# Part 2: Markov Decision Processes and Bellman Equations

Welcome to Part 2 of our Reinforcement Learning tutorial series! In this notebook, we'll formalize the RL problem using **Markov Decision Processes (MDPs)** and derive the fundamental **Bellman equations**.

## What This Notebook Covers
- Markov Processes (Markov Chains) - the simplest building block
- Markov Reward Processes (MRP) - adding rewards to the mix
- Markov Decision Processes (MDP) - the full RL framework
- The Return and discounting future rewards
- Bellman Expectation Equations - recursive value relationships
- Optimal Value Functions and Bellman Optimality Equations

## What This Notebook Does NOT Cover
- Algorithms to solve MDPs (covered in notebook 03: Dynamic Programming)
- Learning from experience without a model (covered in notebooks 04-06)
- Deep reinforcement learning

## Prerequisites
- Completed notebooks 01_1, 01_2, and 01_3
- Basic linear algebra (matrix multiplication, solving linear systems)
- Comfort with mathematical notation ($\sum$, $\mathbb{E}$, etc.)

## Recap from Previous Notebooks

**Notebook 1.1 (Foundations):**
- RL agents learn through trial and error by interacting with an environment
- The Markov property: the future depends only on the present state
- FrozenLake: 4x4 grid with slippery ice, sparse rewards (+1 at Goal only)

**Notebook 1.2 (Policies & Exploration):**
- Policies map states to actions (deterministic or stochastic)
- ε-greedy balances exploration and exploitation
- Prediction evaluates policies; control finds optimal policies

**Notebook 1.3 (Value Functions & Models):**
- Value functions quantify long-term desirability: $V(s)$ and $Q(s,a)$
- The discount factor $\gamma$ weighs future vs immediate rewards
- Models predict transitions $P(s'|s,a)$ and rewards $R(s,a)$

## How to Read This Notebook
1. **Build intuition first**: Each section starts with conceptual explanations using FrozenLake
2. **Then formalize**: We introduce mathematical notation after you understand the concept
3. **Verify understanding**: Checkpoints help you confirm key ideas before moving on
4. **Code demonstrates theory**: Run cells to see the math come alive

Let's begin!

---
## Setup

First, let's import the necessary libraries and set up our environment.

> **Note:** If you're running this in a fresh environment (like Google Colab or a new virtualenv), uncomment and run the installation cell below first.

In [ ]:
# Install necessary libraries (uncomment if needed)
# !pip install gymnasium[toy-text] numpy matplotlib seaborn

In [ ]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output
import time

# Set style for plots
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# Set random seed for reproducibility
np.random.seed(42)

print("Setup complete!")

In [ ]:
# Helper function to visualize the FrozenLake grid
def visualize_frozenlake(env, current_state=None, title="FrozenLake Environment"):
    """Visualize the FrozenLake grid with the current state highlighted."""
    desc = env.unwrapped.desc.astype(str)
    nrow, ncol = desc.shape
    
    fig, ax = plt.subplots(figsize=(6, 6))
    colors = {'S': 'lightblue', 'F': 'white', 'H': 'lightcoral', 'G': 'lightgreen'}
    
    for i in range(nrow):
        for j in range(ncol):
            cell = desc[i, j]
            color = colors.get(cell, 'white')
            state_idx = i * ncol + j
            if current_state is not None and state_idx == current_state:
                rect = plt.Rectangle((j, nrow-1-i), 1, 1, fill=True, 
                                     facecolor='yellow', edgecolor='black', linewidth=2)
            else:
                rect = plt.Rectangle((j, nrow-1-i), 1, 1, fill=True,
                                     facecolor=color, edgecolor='black', linewidth=1)
            ax.add_patch(rect)
            ax.text(j + 0.5, nrow - 1 - i + 0.5, cell,
                   ha='center', va='center', fontsize=20, fontweight='bold')
    
    ax.set_xlim(0, ncol)
    ax.set_ylim(0, nrow)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title(title, fontsize=14, fontweight='bold')
    
    legend_elements = [
        plt.Rectangle((0, 0), 1, 1, facecolor='lightblue', label='S: Start'),
        plt.Rectangle((0, 0), 1, 1, facecolor='white', edgecolor='black', label='F: Frozen (safe)'),
        plt.Rectangle((0, 0), 1, 1, facecolor='lightcoral', label='H: Hole (game over)'),
        plt.Rectangle((0, 0), 1, 1, facecolor='lightgreen', label='G: Goal (reward!)'),
        plt.Rectangle((0, 0), 1, 1, facecolor='yellow', edgecolor='black', linewidth=2, label='Current position')
    ]
    ax.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(1.05, 1))
    plt.tight_layout()
    return fig, ax


def visualize_value_function(V, title="State Value Function V(s)"):
    """Visualize the value function as a heatmap on the FrozenLake grid."""
    V_grid = V.reshape((4, 4))
    
    fig, ax = plt.subplots(figsize=(7, 6))
    
    im = sns.heatmap(V_grid, annot=True, fmt=".3f", cmap="Greens", 
                     cbar_kws={'label': 'Value'}, ax=ax,
                     linewidths=0.5, linecolor='gray')
    
    labels = [['S', 'F', 'F', 'F'],
              ['F', 'H', 'F', 'H'],
              ['F', 'F', 'F', 'H'],
              ['H', 'F', 'F', 'G']]
    
    for i in range(4):
        for j in range(4):
            ax.text(j + 0.5, i + 0.15, labels[i][j], 
                   ha='center', va='center', fontsize=10, 
                   color='red' if labels[i][j] == 'H' else 'blue',
                   fontweight='bold')
    
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.set_xlabel('Column')
    ax.set_ylabel('Row')
    
    plt.tight_layout()
    return fig, ax

---
# 1. Markov Processes (Markov Chains)

Before diving into the full MDP framework, let's build up from simpler structures. A **Markov Process** (also called a Markov Chain) is the simplest model of sequential state transitions.

## Definition

A **Markov Process** is a tuple $(S, P)$ where:
- $S$ is a finite set of states
- $P$ is a **state transition probability matrix**: $P_{ss'} = P[S_{t+1} = s' | S_t = s]$

The Markov Process describes a random walk through states, where the next state depends **only on the current state** (the Markov property we learned in Notebook 1.1).

## Key Properties

| Property | Description |
|----------|-------------|
| **No actions** | The agent doesn't make decisions - transitions happen automatically |
| **No rewards** | We only track which states we visit, not how "good" they are |
| **Memoryless** | Future states depend only on the current state, not history |
| **Stochastic** | Transitions are probabilistic (each row of $P$ sums to 1) |

## FrozenLake as a Markov Process

If we take FrozenLake and fix a policy (say, uniform random), the agent no longer makes decisions - it just follows the policy automatically. This gives us a Markov Process!

- **States**: The 16 positions on the grid (0-15)
- **Transitions**: Determined by the fixed policy + slippery ice dynamics

In [ ]:
# Create FrozenLake environment
env = gym.make("FrozenLake-v1", render_mode="rgb_array", is_slippery=True)
env.reset(seed=42)

print("FrozenLake with Random Policy = Markov Process")
print("=" * 50)
print(f"\nStates S: {{{', '.join(map(str, range(16)))}}}")
print(f"Number of states |S|: {env.observation_space.n}")
print("\nWith a fixed random policy, the agent doesn't choose -")
print("it just follows the policy, creating a Markov Chain.")

In [ ]:
# Build the transition matrix P for FrozenLake under a uniform random policy
def build_markov_chain_matrix(env):
    """
    Build transition matrix P for FrozenLake with uniform random policy.
    P[s, s'] = probability of going from state s to state s'
    """
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    
    # Under uniform random policy, each action has probability 1/n_actions
    policy_prob = 1.0 / n_actions
    
    # Initialize transition matrix
    P = np.zeros((n_states, n_states))
    
    # For each state and action, accumulate transition probabilities
    for state in range(n_states):
        for action in range(n_actions):
            # Get transitions for this state-action pair
            for prob, next_state, reward, done in env.unwrapped.P[state][action]:
                # Weight by policy probability
                P[state, next_state] += policy_prob * prob
    
    return P

P_chain = build_markov_chain_matrix(env)

print("Transition Matrix P (Markov Chain under Random Policy)")
print("=" * 55)
print(f"\nShape: {P_chain.shape} (16 states × 16 states)")
print(f"Row sums (should all be 1.0): {P_chain.sum(axis=1)[:4]}... (first 4 shown)")
print("\nTransition probabilities FROM state 0 (Start):")
for next_s in range(16):
    if P_chain[0, next_s] > 0:
        print(f"  P(0 → {next_s}) = {P_chain[0, next_s]:.4f}")

In [ ]:
# Visualize the transition matrix as a heatmap
print("**Question this plot answers:** 'What are the transition probabilities between all states?'\n")

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(P_chain, annot=False, cmap='Blues', ax=ax,
            xticklabels=range(16), yticklabels=range(16))
ax.set_xlabel('Next State s\'')
ax.set_ylabel('Current State s')
ax.set_title('Markov Chain Transition Matrix P\n(FrozenLake with Random Policy)')
plt.tight_layout()
plt.show()

print("Notice: Terminal states (5, 7, 11, 12 = Holes, 15 = Goal) have self-loops")
print("(probability 1.0 of staying in the same state).")

In [ ]:
# Simulate random walks on the Markov Chain
def simulate_markov_chain(P, start_state, max_steps=50):
    """Simulate a trajectory through the Markov Chain."""
    trajectory = [start_state]
    current = start_state
    terminal_states = {5, 7, 11, 12, 15}  # Holes and Goal
    
    for _ in range(max_steps):
        if current in terminal_states:
            break
        # Sample next state according to transition probabilities
        next_state = np.random.choice(len(P), p=P[current])
        trajectory.append(next_state)
        current = next_state
    
    return trajectory

# Simulate several episodes
np.random.seed(42)
print("Sample Trajectories through Markov Chain (starting from state 0)")
print("=" * 60)

for episode in range(5):
    traj = simulate_markov_chain(P_chain, start_state=0)
    final_state = traj[-1]
    outcome = "Goal!" if final_state == 15 else f"Hole at {final_state}" if final_state in {5,7,11,12} else "Timeout"
    print(f"\nEpisode {episode + 1}: {' → '.join(map(str, traj))}")
    print(f"   Outcome: {outcome} ({len(traj)} steps)")

> **Checkpoint — You should now understand:**
> - A Markov Process (Markov Chain) is defined by states $S$ and transitions $P$
> - With a fixed policy, an MDP becomes a Markov Chain
> - The transition matrix $P$ captures all state-to-state probabilities
> - Each row of $P$ sums to 1 (we must go somewhere)

---
# 2. Markov Reward Process (MRP)

A **Markov Reward Process** adds rewards to a Markov Chain. Now we care not just about *where* we go, but *how good* each state is.

## Definition

An MRP is a tuple $(S, P, R, \gamma)$ where:
- $S$ is a finite set of states
- $P$ is the state transition probability matrix
- $R$ is a **reward function**: $R_s = \mathbb{E}[R_{t+1} | S_t = s]$ (expected reward in state $s$)
- $\gamma$ is a **discount factor**, $\gamma \in [0, 1]$

## Building an MRP from FrozenLake

When we combine FrozenLake with a fixed policy, we get an MRP:

| MRP Component | FrozenLake (Random Policy) |
|---------------|---------------------------|
| States $S$ | 16 grid positions |
| Transitions $P$ | From random policy + slippery ice |
| Rewards $R$ | +1 at Goal (state 15), 0 elsewhere |
| Discount $\gamma$ | Typically 0.99 |

In [ ]:
# Build the MRP from FrozenLake with random policy
def build_mrp_reward_vector(env):
    """
    Build reward vector R for FrozenLake with uniform random policy.
    R[s] = expected immediate reward when in state s
    """
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    policy_prob = 1.0 / n_actions
    
    R = np.zeros(n_states)
    
    for state in range(n_states):
        for action in range(n_actions):
            for prob, next_state, reward, done in env.unwrapped.P[state][action]:
                R[state] += policy_prob * prob * reward
    
    return R

R_mrp = build_mrp_reward_vector(env)

print("MRP Reward Vector R (Expected Immediate Reward)")
print("=" * 50)
print("\nState  Expected Reward  Interpretation")
print("-" * 50)
for s in range(16):
    desc = "Start" if s == 0 else "Goal" if s == 15 else "Hole" if s in {5,7,11,12} else "Frozen"
    print(f"  {s:2d}      {R_mrp[s]:.4f}         {desc}")

## The Return $G_t$

The **return** $G_t$ is the total discounted reward from time step $t$ onwards:

$$G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \ldots = \sum_{k=0}^{\infty} \gamma^k R_{t+k+1}$$

### Why Discount?

| Reason | Explanation | FrozenLake Example |
|--------|-------------|-------------------|
| **Uncertainty** | Future is uncertain; prefer certain rewards now | Slippery ice might cause failure later |
| **Preference** | Humans/animals prefer sooner rewards | Reaching Goal in 5 steps > 20 steps |
| **Math** | Prevents infinite returns in continuing tasks | Ensures $G_t$ is finite even if episode never ends |

### Effect of $\gamma$

| $\gamma$ | Behavior | Reaching Goal in 10 steps gives... |
|----------|----------|------------------------------------|
| 0 | Only immediate reward matters | $G = 0$ (no immediate reward at step 0) |
| 0.9 | Future matters but is discounted | $G = 0.9^{10} \times 1 = 0.35$ |
| 0.99 | Future almost as important | $G = 0.99^{10} \times 1 = 0.90$ |
| 1.0 | All rewards equally important | $G = 1.0$ |

In [ ]:
# Compute returns from sample episodes
def compute_return(rewards, gamma):
    """Compute discounted return from a sequence of rewards."""
    G = 0
    for t in reversed(range(len(rewards))):
        G = rewards[t] + gamma * G
    return G

def run_mrp_episode(env, max_steps=100):
    """Run one episode with random policy, return trajectory and rewards."""
    obs, _ = env.reset()
    trajectory = [obs]
    rewards = []
    
    for _ in range(max_steps):
        action = env.action_space.sample()
        obs, reward, terminated, truncated, _ = env.step(action)
        rewards.append(reward)
        trajectory.append(obs)
        
        if terminated or truncated:
            break
    
    return trajectory, rewards

# Run sample episodes and compute returns
np.random.seed(42)
gamma = 0.99

print(f"Sample Episodes and Returns (γ = {gamma})")
print("=" * 60)

for episode in range(5):
    traj, rewards = run_mrp_episode(env)
    G = compute_return(rewards, gamma)
    outcome = "Goal!" if traj[-1] == 15 else f"Hole" if traj[-1] in {5,7,11,12} else "Timeout"
    
    print(f"\nEpisode {episode + 1}:")
    print(f"  Path: {' → '.join(map(str, traj[:8]))}{'...' if len(traj) > 8 else ''}")
    print(f"  Steps: {len(rewards)}, Outcome: {outcome}")
    print(f"  Rewards: {rewards}")
    print(f"  Return G₀ = {G:.4f}")

## Value Function for MRP

The **state value function** $v(s)$ gives the expected return starting from state $s$:

$$v(s) = \mathbb{E}[G_t | S_t = s]$$

This single number captures the **long-term desirability** of being in state $s$.

## The Bellman Equation for MRP

The value function can be decomposed into two parts:
1. **Immediate reward** $R_s$
2. **Discounted value of successor states** $\gamma \sum_{s'} P_{ss'} v(s')$

$$v(s) = R_s + \gamma \sum_{s' \in S} P_{ss'} v(s')$$

In plain English: *"The value of being here = what I get now + what I expect to get later (discounted)"*

### Matrix Form

The Bellman equation can be written as:

$$v = R + \gamma P v$$

This is a **linear equation** that can be solved directly:

$$(I - \gamma P) v = R$$
$$v = (I - \gamma P)^{-1} R$$

In [ ]:
# Solve the Bellman equation for MRP directly
def solve_mrp_bellman(P, R, gamma):
    """Solve MRP Bellman equation: v = (I - gamma*P)^(-1) * R"""
    n = len(R)
    I = np.eye(n)
    V = np.linalg.solve(I - gamma * P, R)
    return V

# Solve for value function
gamma = 0.99
V_mrp = solve_mrp_bellman(P_chain, R_mrp, gamma)

print(f"MRP State Values (Solved via Bellman Equation, γ = {gamma})")
print("=" * 55)
print("\nState   V(s)      Interpretation")
print("-" * 55)
for s in range(16):
    desc = "Start" if s == 0 else "Goal" if s == 15 else "Hole" if s in {5,7,11,12} else "Frozen"
    print(f"  {s:2d}    {V_mrp[s]:.4f}    {desc}")

In [ ]:
# Visualize the MRP value function
print("**Question this heatmap answers:** 'How valuable is each state under random policy?'\n")

visualize_value_function(V_mrp, title=f"MRP Value Function (Random Policy, γ={gamma})")
plt.show()

print("\nObservations:")
print("- Holes and Goal have V=0 (terminal states, no future rewards)")
print("- States closer to Goal have higher values")
print("- Values are low overall because random policy rarely reaches Goal")

In [ ]:
# Verify the Bellman equation for a specific state
state = 10  # A non-terminal state near the Goal

print(f"Verifying Bellman Equation for State {state}")
print("=" * 55)
print(f"\nBellman equation: v(s) = R_s + γ × Σ P(s'|s) × v(s')")
print(f"\nFor state {state}:")
print(f"  R_{state} = {R_mrp[state]:.6f}")
print(f"  γ = {gamma}")

# Calculate expected value of successor states
expected_next_value = 0
print(f"\n  Expected next state value:")
for next_s in range(16):
    if P_chain[state, next_s] > 0:
        contribution = P_chain[state, next_s] * V_mrp[next_s]
        expected_next_value += contribution
        print(f"    P({next_s}|{state}) × V({next_s}) = {P_chain[state, next_s]:.3f} × {V_mrp[next_s]:.4f} = {contribution:.6f}")

print(f"\n  Σ P(s'|s) × V(s') = {expected_next_value:.6f}")

# Calculate total using Bellman equation
v_calculated = R_mrp[state] + gamma * expected_next_value
print(f"\n  v({state}) = {R_mrp[state]:.6f} + {gamma} × {expected_next_value:.6f} = {v_calculated:.6f}")
print(f"  Direct solution: V({state}) = {V_mrp[state]:.6f}")
print(f"\n  ✓ Bellman equation verified!" if abs(v_calculated - V_mrp[state]) < 1e-6 else "  ✗ Error!")

> **Checkpoint — You should now understand:**
> - An MRP adds rewards and discounting to a Markov Chain: $(S, P, R, \gamma)$
> - The return $G_t$ is the sum of discounted future rewards
> - The value function $v(s) = \mathbb{E}[G_t | S_t = s]$ captures long-term desirability
> - The Bellman equation relates $v(s)$ to immediate reward + discounted future value
> - We can solve for $v$ directly using matrix inversion

---
# 3. Markov Decision Process (MDP)

A **Markov Decision Process** adds **actions** to the MRP. Now the agent can **choose** what to do, rather than following a fixed behavior.

This is the full framework for reinforcement learning!

## Definition

An MDP is a tuple $(S, A, P, R, \gamma)$ where:
- $S$ is a finite set of **states**
- $A$ is a finite set of **actions**
- $P$ is the state transition probability: $P^a_{ss'} = P[S_{t+1} = s' | S_t = s, A_t = a]$
- $R$ is the reward function: $R^a_s = \mathbb{E}[R_{t+1} | S_t = s, A_t = a]$
- $\gamma$ is a discount factor, $\gamma \in [0, 1]$

## Key Difference from MRP

| Aspect | MRP | MDP |
|--------|-----|-----|
| **Actions** | None (transitions happen automatically) | Agent chooses actions |
| **Transitions** | $P_{ss'}$ (state to state) | $P^a_{ss'}$ (depends on action) |
| **Rewards** | $R_s$ (depends on state) | $R^a_s$ (depends on state AND action) |
| **Agent's role** | Passive observer | Active decision maker |

In [ ]:
# FrozenLake as an MDP
env = gym.make("FrozenLake-v1", render_mode="rgb_array", is_slippery=True)
env.reset(seed=42)

print("FrozenLake MDP Components")
print("=" * 50)
print(f"\nS (States):  {env.observation_space}  (16 grid positions)")
print(f"A (Actions): {env.action_space}  (4 directions)")
print(f"\nActions: 0=LEFT, 1=DOWN, 2=RIGHT, 3=UP")
print(f"\nγ (Discount): Typically 0.99")

# Show the map
print("\nEnvironment Map:")
desc = env.unwrapped.desc.astype(str)
for i, row in enumerate(desc):
    states = [str(i*4 + j).rjust(2) for j in range(4)]
    print(f"   {' '.join(row)}    (states {', '.join(states)})")

In [ ]:
# 🔍 INSTRUCTOR-ONLY: Extract MDP transition and reward matrices
# A real RL agent would NOT have access to this - it would learn through experience!

def extract_mdp_components(env):
    """Extract P and R from a Gymnasium environment.
    
    Returns:
        T: Transition tensor T[s, a, s'] = P(s'|s, a)
        R: Reward matrix R[s, a] = expected reward
    """
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    
    T = np.zeros((n_states, n_actions, n_states))
    R = np.zeros((n_states, n_actions))
    
    for s in range(n_states):
        for a in range(n_actions):
            for prob, next_s, reward, done in env.unwrapped.P[s][a]:
                T[s, a, next_s] += prob
                R[s, a] += prob * reward
    
    return T, R

T_mdp, R_mdp = extract_mdp_components(env)

print("🔍 INSTRUCTOR-ONLY: MDP Transition and Reward Tensors")
print("=" * 55)
print("\n⚠️  A real RL agent does NOT have access to these!")
print("    We use them here for educational demonstration only.")
print(f"\nTransition tensor T[s,a,s'] shape: {T_mdp.shape}")
print(f"Reward matrix R[s,a] shape: {R_mdp.shape}")

In [ ]:
# 🔍 INSTRUCTOR-ONLY: Examine transitions from a specific state
action_names = {0: 'LEFT', 1: 'DOWN', 2: 'RIGHT', 3: 'UP'}
state = 6

print(f"🔍 INSTRUCTOR-ONLY: Transitions from State {state}")
print("=" * 55)
print(f"\nState {state} is in the middle of the grid (row 1, col 2)")

for action in range(4):
    print(f"\n  Action {action} ({action_names[action]}):")
    print(f"    Expected reward: R({state},{action}) = {R_mdp[state, action]:.4f}")
    print(f"    Possible next states:")
    for next_s in range(16):
        if T_mdp[state, action, next_s] > 0:
            cell_type = desc.flatten()[next_s]
            print(f"      → State {next_s} ({cell_type}): P = {T_mdp[state, action, next_s]:.2f}")

In [ ]:
# Visualize transition probabilities from one state for all actions
print("**Question this plot answers:** 'How does each action affect where I might end up?'\n")

def visualize_mdp_transitions(state, T, env):
    """Visualize transitions from a state for all actions."""
    desc = env.unwrapped.desc.astype(str)
    nrow, ncol = desc.shape
    action_names = ['LEFT', 'DOWN', 'RIGHT', 'UP']
    colors = {'S': 'lightblue', 'F': 'white', 'H': 'lightcoral', 'G': 'lightgreen'}
    
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    
    for action, ax in enumerate(axes):
        for i in range(nrow):
            for j in range(ncol):
                cell = desc[i, j]
                state_idx = i * ncol + j
                prob = T[state, action, state_idx]
                
                if state_idx == state:
                    facecolor = 'yellow'
                elif prob > 0:
                    facecolor = plt.cm.Blues(0.3 + 0.7 * prob)
                else:
                    facecolor = colors.get(cell, 'white')
                
                rect = plt.Rectangle((j, nrow-1-i), 1, 1, fill=True,
                                     facecolor=facecolor, edgecolor='black')
                ax.add_patch(rect)
                
                ax.text(j + 0.5, nrow - 1 - i + 0.7, cell,
                       ha='center', va='center', fontsize=12)
                
                if prob > 0 and state_idx != state:
                    ax.text(j + 0.5, nrow - 1 - i + 0.3, f'{prob:.2f}',
                           ha='center', va='center', fontsize=10, 
                           color='blue', fontweight='bold')
        
        ax.set_xlim(0, ncol)
        ax.set_ylim(0, nrow)
        ax.set_aspect('equal')
        ax.axis('off')
        ax.set_title(f'Action: {action_names[action]}')
    
    plt.suptitle(f'Transition Probabilities from State {state} (Yellow)\n'
                 f'Blue intensity = probability, numbers show P(s\'|s,a)', y=1.05)
    plt.tight_layout()
    return fig

visualize_mdp_transitions(6, T_mdp, env)
plt.show()

print("Notice: Due to slippery ice, the intended action only succeeds 1/3 of the time!")
print("The other 2/3, the agent slips perpendicular to the intended direction.")

> **Checkpoint — You should now understand:**
> - An MDP adds actions to an MRP: $(S, A, P, R, \gamma)$
> - Transitions and rewards now depend on both state AND action
> - The agent actively chooses actions rather than following fixed behavior
> - In FrozenLake, the slippery ice makes transitions stochastic even with a chosen action

---
# 4. Policies in MDPs

A **policy** $\pi$ defines the agent's behavior - how it chooses actions based on states.

$$\pi(a|s) = P[A_t = a | S_t = s]$$

## Types of Policies

| Type | Notation | Description | Example |
|------|----------|-------------|---------|
| **Deterministic** | $a = \pi(s)$ | Always same action for a state | "In state 6, always go RIGHT" |
| **Stochastic** | $\pi(a|s)$ | Probability distribution over actions | "In state 6: 40% RIGHT, 30% DOWN, ..." |

## Key Insight: MDP + Policy = MRP

Once we fix a policy $\pi$, the MDP becomes an MRP!

- **Transitions**: $P^\pi_{ss'} = \sum_a \pi(a|s) P^a_{ss'}$
- **Rewards**: $R^\pi_s = \sum_a \pi(a|s) R^a_s$

The policy "averages out" the action choice, leaving us with state-to-state transitions.

In [ ]:
# Define some policies for FrozenLake
n_states = env.observation_space.n
n_actions = env.action_space.n

# Uniform random policy
def create_uniform_policy():
    """Return a uniform random policy."""
    return np.ones((n_states, n_actions)) / n_actions

# "Go DOWN then RIGHT" policy
def create_down_right_policy():
    """Policy that prefers DOWN in upper half, RIGHT in lower half."""
    policy = np.zeros((n_states, n_actions))
    for s in range(n_states):
        row = s // 4
        if row < 2:  # Upper half: go DOWN
            policy[s, 1] = 1.0  # DOWN
        else:  # Lower half: go RIGHT
            policy[s, 2] = 1.0  # RIGHT
    return policy

pi_random = create_uniform_policy()
pi_down_right = create_down_right_policy()

print("Example Policies for FrozenLake")
print("=" * 50)

print("\nUniform Random Policy π_random(a|s=6):")
for a in range(n_actions):
    print(f"  π({action_names[a]}|s=6) = {pi_random[6, a]:.2f}")

print("\nDown-then-Right Policy π_dr(a|s=6):")
for a in range(n_actions):
    print(f"  π({action_names[a]}|s=6) = {pi_down_right[6, a]:.2f}")

In [ ]:
# Convert MDP + Policy to MRP
def mdp_to_mrp(T, R_sa, policy):
    """
    Convert MDP with policy to MRP.
    
    Args:
        T: Transition tensor T[s,a,s']
        R_sa: Reward matrix R[s,a]
        policy: Policy matrix π[s,a]
    
    Returns:
        P_mrp: MRP transition matrix P[s,s']
        R_mrp: MRP reward vector R[s]
    """
    n_states = T.shape[0]
    n_actions = T.shape[1]
    
    # P_ss'^π = Σ_a π(a|s) × P_ss'^a
    P_mrp = np.zeros((n_states, n_states))
    for s in range(n_states):
        for a in range(n_actions):
            P_mrp[s] += policy[s, a] * T[s, a]
    
    # R_s^π = Σ_a π(a|s) × R_s^a
    R_mrp = np.sum(policy * R_sa, axis=1)
    
    return P_mrp, R_mrp

# Convert with random policy
P_from_random, R_from_random = mdp_to_mrp(T_mdp, R_mdp, pi_random)
P_from_dr, R_from_dr = mdp_to_mrp(T_mdp, R_mdp, pi_down_right)

print("MDP + Policy → MRP")
print("=" * 50)
print(f"\nMDP: T shape = {T_mdp.shape}, R shape = {R_mdp.shape}")
print(f"\nWith Random Policy:")
print(f"  MRP: P shape = {P_from_random.shape}, R shape = {R_from_random.shape}")
print(f"\nWith Down-Right Policy:")
print(f"  MRP: P shape = {P_from_dr.shape}, R shape = {R_from_dr.shape}")
print(f"\nRow sums of P (should be 1): {P_from_random.sum(axis=1)[:4]}...")

> **Checkpoint — You should now understand:**
> - A policy $\pi(a|s)$ defines how the agent chooses actions
> - Policies can be deterministic or stochastic
> - MDP + fixed policy = MRP (the policy "collapses" the action dimension)

---
# 5. Value Functions for MDPs

In MDPs, we have two value functions that depend on the policy:

## State-Value Function $V^\pi(s)$

$$V^\pi(s) = \mathbb{E}_\pi[G_t | S_t = s]$$

**"How good is it to BE in state $s$ if I follow policy $\pi$?"**

## Action-Value Function $Q^\pi(s, a)$

$$Q^\pi(s, a) = \mathbb{E}_\pi[G_t | S_t = s, A_t = a]$$

**"How good is it to DO action $a$ in state $s$, then follow policy $\pi$?"**

## Comparison

| Aspect | $V^\pi(s)$ | $Q^\pi(s, a)$ |
|--------|-----------|---------------|
| **Input** | State only | State AND action |
| **Question** | "How good is this state?" | "How good is this action here?" |
| **Size (FrozenLake)** | 16 values | 64 values (16 × 4) |
| **Use** | Evaluate being in a state | Compare different actions |

In [ ]:
# Compute V^π using MRP solution
def compute_V_pi(T, R_sa, policy, gamma):
    """Compute V^π using direct solution of Bellman equation."""
    P_mrp, R_mrp = mdp_to_mrp(T, R_sa, policy)
    n_states = len(R_mrp)
    I = np.eye(n_states)
    V = np.linalg.solve(I - gamma * P_mrp, R_mrp)
    return V

# Compute Q^π from V^π
def compute_Q_from_V(T, R_sa, V, gamma):
    """Compute Q^π from V^π."""
    n_states, n_actions = R_sa.shape
    Q = np.zeros((n_states, n_actions))
    
    for s in range(n_states):
        for a in range(n_actions):
            # Q(s,a) = R(s,a) + γ × Σ P(s'|s,a) × V(s')
            Q[s, a] = R_sa[s, a] + gamma * np.sum(T[s, a] * V)
    
    return Q

# Compute values for random policy
gamma = 0.99
V_random = compute_V_pi(T_mdp, R_mdp, pi_random, gamma)
Q_random = compute_Q_from_V(T_mdp, R_mdp, V_random, gamma)

print(f"Value Functions for Random Policy (γ = {gamma})")
print("=" * 55)
print("\nState-Value Function V^π(s):")
print(V_random.reshape(4, 4).round(4))

In [ ]:
# Visualize V^π
print("**Question this heatmap answers:** 'Under random policy, which states are most valuable?'\n")

visualize_value_function(V_random, title=f"V^π (Random Policy, γ={gamma})")
plt.show()

In [ ]:
# Visualize Q^π values
print("**Question this plot answers:** 'Which action is best in each state under random policy?'\n")

def visualize_q_values(Q, env, title="Q-Values"):
    """Visualize Q-values showing best action in each state."""
    desc = env.unwrapped.desc.astype(str)
    nrow, ncol = desc.shape
    action_arrows = ['←', '↓', '→', '↑']
    colors = {'S': 'lightblue', 'F': 'white', 'H': 'lightcoral', 'G': 'lightgreen'}
    
    fig, ax = plt.subplots(figsize=(10, 10))
    
    for i in range(nrow):
        for j in range(ncol):
            state = i * ncol + j
            cell = desc[i, j]
            
            rect = plt.Rectangle((j, nrow-1-i), 1, 1, fill=True,
                                 facecolor=colors.get(cell, 'white'), edgecolor='black')
            ax.add_patch(rect)
            
            ax.text(j + 0.5, nrow - 1 - i + 0.5, cell, ha='center', va='center',
                   fontsize=14, fontweight='bold', alpha=0.3)
            
            if cell not in ['H', 'G']:
                best_action = np.argmax(Q[state])
                positions = [(0.2, 0.5), (0.5, 0.2), (0.8, 0.5), (0.5, 0.8)]  # L, D, R, U
                for a in range(4):
                    x, y = positions[a]
                    color = 'green' if a == best_action else 'black'
                    weight = 'bold' if a == best_action else 'normal'
                    ax.text(j + x, nrow - 1 - i + y, f'{Q[state, a]:.2f}',
                           ha='center', va='center', fontsize=7, 
                           color=color, fontweight=weight)
    
    ax.set_xlim(0, ncol)
    ax.set_ylim(0, nrow)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title(f"{title}\n(Green = best action, positions: ←left, ↓bottom, →right, ↑top)", fontsize=12)
    
    plt.tight_layout()
    return fig, ax

visualize_q_values(Q_random, env, title="Q^π (Random Policy)")
plt.show()

## Relationship Between V and Q

The two value functions are related:

$$V^\pi(s) = \sum_a \pi(a|s) \cdot Q^\pi(s, a)$$

**V is the weighted average of Q values, weighted by the policy.**

For a **deterministic policy** that always chooses action $a^*$:
$$V^\pi(s) = Q^\pi(s, a^*)$$

For a **uniform random policy**:
$$V^\pi(s) = \frac{1}{|A|} \sum_a Q^\pi(s, a)$$

In [ ]:
# Verify V = Σ π(a|s) × Q(s,a)
state = 10

print(f"Verifying V = Σ π(a|s) × Q(s,a) for State {state}")
print("=" * 55)

V_direct = V_random[state]
print(f"\nDirect V^π({state}) = {V_direct:.6f}")

print(f"\nUsing V = Σ π(a|s) × Q(s,a):")
V_from_Q = 0
for a in range(n_actions):
    contribution = pi_random[state, a] * Q_random[state, a]
    V_from_Q += contribution
    print(f"  π({action_names[a]}|{state}) × Q({state},{action_names[a]}) = {pi_random[state,a]:.2f} × {Q_random[state,a]:.4f} = {contribution:.6f}")

print(f"\nSum = {V_from_Q:.6f}")
print(f"\n✓ Verified!" if abs(V_direct - V_from_Q) < 1e-6 else "✗ Error!")

> **Checkpoint — You should now understand:**
> - $V^\pi(s)$: expected return starting from state $s$ under policy $\pi$
> - $Q^\pi(s, a)$: expected return after taking action $a$ in state $s$, then following $\pi$
> - $V^\pi(s) = \sum_a \pi(a|s) Q^\pi(s, a)$ — V is the policy-weighted average of Q

---
# 6. Bellman Expectation Equations

The Bellman equations express the **recursive relationship** between value functions.

## Intuition: Value = Immediate + Future

The value of a state (or state-action pair) can be decomposed:

**Value now = What I get now + (Discounted) What I expect later**

## Bellman Expectation Equation for $V^\pi$

$$V^\pi(s) = \sum_a \pi(a|s) \left[ R^a_s + \gamma \sum_{s'} P^a_{ss'} V^\pi(s') \right]$$

Breaking this down:
1. $\sum_a \pi(a|s)$ — average over actions according to policy
2. $R^a_s$ — immediate reward for taking action $a$
3. $\gamma \sum_{s'} P^a_{ss'} V^\pi(s')$ — discounted expected value of next states

## Bellman Expectation Equation for $Q^\pi$

$$Q^\pi(s, a) = R^a_s + \gamma \sum_{s'} P^a_{ss'} V^\pi(s')$$

Or equivalently, substituting $V$:

$$Q^\pi(s, a) = R^a_s + \gamma \sum_{s'} P^a_{ss'} \sum_{a'} \pi(a'|s') Q^\pi(s', a')$$

In [ ]:
# Verify Bellman Expectation Equation for V^π
state = 10
gamma = 0.99

print(f"Verifying Bellman Expectation Equation for V^π at State {state}")
print("=" * 65)
print(f"\nEquation: V^π(s) = Σ_a π(a|s) × [R(s,a) + γ × Σ_s' P(s'|s,a) × V^π(s')]")

V_direct = V_random[state]
print(f"\nDirect solution: V^π({state}) = {V_direct:.6f}")

print(f"\nUsing Bellman equation:")
V_bellman = 0
for a in range(n_actions):
    # Immediate reward + discounted future value
    immediate = R_mdp[state, a]
    future = gamma * np.sum(T_mdp[state, a] * V_random)
    q_sa = immediate + future
    
    contribution = pi_random[state, a] * q_sa
    V_bellman += contribution
    
    print(f"  a={action_names[a]}: π={pi_random[state,a]:.2f} × [R={immediate:.4f} + γ×ΣPV={future:.4f}] = {contribution:.6f}")

print(f"\nSum = {V_bellman:.6f}")
print(f"\n✓ Bellman equation verified!" if abs(V_direct - V_bellman) < 1e-6 else "✗ Error!")

In [ ]:
# Verify Bellman Expectation Equation for Q^π
state = 10
action = 2  # RIGHT

print(f"Verifying Bellman Expectation Equation for Q^π at State {state}, Action {action_names[action]}")
print("=" * 70)
print(f"\nEquation: Q^π(s,a) = R(s,a) + γ × Σ_s' P(s'|s,a) × V^π(s')")

Q_direct = Q_random[state, action]
print(f"\nDirect solution: Q^π({state},{action_names[action]}) = {Q_direct:.6f}")

print(f"\nUsing Bellman equation:")
print(f"  R({state},{action_names[action]}) = {R_mdp[state, action]:.6f}")

future_value = 0
print(f"\n  γ × Σ P(s'|s,a) × V^π(s'):")
for next_s in range(n_states):
    if T_mdp[state, action, next_s] > 0:
        contribution = T_mdp[state, action, next_s] * V_random[next_s]
        future_value += contribution
        print(f"    P({next_s}|{state},{action_names[action]}) × V({next_s}) = {T_mdp[state,action,next_s]:.2f} × {V_random[next_s]:.4f} = {contribution:.6f}")

print(f"\n  Σ = {future_value:.6f}")
print(f"  γ × Σ = {gamma * future_value:.6f}")

Q_bellman = R_mdp[state, action] + gamma * future_value
print(f"\nQ^π({state},{action_names[action]}) = {R_mdp[state,action]:.6f} + {gamma * future_value:.6f} = {Q_bellman:.6f}")
print(f"\n✓ Bellman equation verified!" if abs(Q_direct - Q_bellman) < 1e-6 else "✗ Error!")

> **Checkpoint — You should now understand:**
> - Bellman equations express value recursively: Value = Immediate + γ × Expected Future
> - For $V^\pi$: average over actions, then over next states
> - For $Q^\pi$: immediate reward + discounted expected next state value
> - These equations are the foundation for RL algorithms

---
# 7. Optimal Value Functions and Bellman Optimality

So far, we've evaluated specific policies. Now let's find the **best** policy!

## Optimal Value Functions

The **optimal state-value function** $V^*(s)$ is the maximum value achievable:

$$V^*(s) = \max_\pi V^\pi(s)$$

The **optimal action-value function** $Q^*(s, a)$ is the maximum action-value:

$$Q^*(s, a) = \max_\pi Q^\pi(s, a)$$

## Key Theorem

For any finite MDP:
1. There exists an **optimal policy** $\pi^*$ that is better than or equal to all other policies
2. All optimal policies achieve the same $V^*$ and $Q^*$
3. There always exists a **deterministic** optimal policy

## Finding Optimal Policy from $Q^*$

If we know $Q^*$, the optimal policy is simple:

$$\pi^*(s) = \arg\max_a Q^*(s, a)$$

**Just pick the action with the highest Q-value!**

## Bellman Optimality Equations

The optimal value functions satisfy special Bellman equations:

### For $V^*$:

$$V^*(s) = \max_a \left[ R^a_s + \gamma \sum_{s'} P^a_{ss'} V^*(s') \right]$$

### For $Q^*$:

$$Q^*(s, a) = R^a_s + \gamma \sum_{s'} P^a_{ss'} \max_{a'} Q^*(s', a')$$

### Key Difference from Bellman Expectation Equations

| Aspect | Expectation Equations | Optimality Equations |
|--------|----------------------|---------------------|
| **Operator** | $\sum_a \pi(a|s)$ (average) | $\max_a$ (maximum) |
| **Linearity** | Linear in $V$ | **Non-linear** (due to max) |
| **Solution** | Matrix inversion | Iterative methods required |

In [ ]:
# Value Iteration to find V* (preview of notebook 03)
def value_iteration(T, R, gamma, theta=1e-8, max_iterations=1000):
    """Find optimal value function using Value Iteration."""
    n_states = T.shape[0]
    n_actions = T.shape[1]
    
    V = np.zeros(n_states)
    
    for iteration in range(max_iterations):
        V_new = np.zeros(n_states)
        
        for s in range(n_states):
            # V*(s) = max_a [R(s,a) + γ × Σ P(s'|s,a) × V*(s')]
            q_values = np.zeros(n_actions)
            for a in range(n_actions):
                q_values[a] = R[s, a] + gamma * np.sum(T[s, a] * V)
            V_new[s] = np.max(q_values)
        
        # Check convergence
        if np.max(np.abs(V_new - V)) < theta:
            print(f"Value Iteration converged after {iteration + 1} iterations")
            break
        
        V = V_new
    
    return V

# Find optimal value function
V_star = value_iteration(T_mdp, R_mdp, gamma=0.99)

# Compute Q* from V*
Q_star = compute_Q_from_V(T_mdp, R_mdp, V_star, gamma=0.99)

# Extract optimal policy
pi_star = np.zeros((n_states, n_actions))
for s in range(n_states):
    best_action = np.argmax(Q_star[s])
    pi_star[s, best_action] = 1.0

print("\nOptimal Value Function V*:")
print(V_star.reshape(4, 4).round(4))

In [ ]:
# Visualize optimal values and policy
print("**Question these plots answer:** 'What is the optimal value and policy for FrozenLake?'\n")

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Plot V*
desc = env.unwrapped.desc.astype(str)
nrow, ncol = desc.shape
V_grid = V_star.reshape(nrow, ncol)

im = axes[0].imshow(V_grid, cmap='Greens', vmin=0, vmax=V_star.max())
plt.colorbar(im, ax=axes[0], label='V*(s)')

for i in range(nrow):
    for j in range(ncol):
        state = i * ncol + j
        axes[0].text(j, i, f'{desc[i,j]}\n{V_star[state]:.3f}', 
                    ha='center', va='center', fontsize=10)

axes[0].set_title('Optimal Value Function V*', fontsize=12)
axes[0].set_xticks(range(ncol))
axes[0].set_yticks(range(nrow))

# Plot optimal policy
action_arrows = ['←', '↓', '→', '↑']
colors = {'S': 'lightblue', 'F': 'white', 'H': 'lightcoral', 'G': 'lightgreen'}

for i in range(nrow):
    for j in range(ncol):
        state = i * ncol + j
        cell = desc[i, j]
        
        rect = plt.Rectangle((j, nrow-1-i), 1, 1, fill=True,
                             facecolor=colors.get(cell, 'white'), edgecolor='black')
        axes[1].add_patch(rect)
        
        best_action = np.argmax(Q_star[state])
        if cell not in ['H', 'G']:
            axes[1].text(j + 0.5, nrow - 1 - i + 0.5, 
                        f'{cell}\n{action_arrows[best_action]}',
                        ha='center', va='center', fontsize=14, fontweight='bold')
        else:
            axes[1].text(j + 0.5, nrow - 1 - i + 0.5, cell,
                        ha='center', va='center', fontsize=14, fontweight='bold')

axes[1].set_xlim(0, ncol)
axes[1].set_ylim(0, nrow)
axes[1].set_aspect('equal')
axes[1].axis('off')
axes[1].set_title('Optimal Policy π*\n(arrows show best action)', fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
# Compare random policy vs optimal policy performance
print("**Question this comparison answers:** 'How much better is the optimal policy?'\n")

def evaluate_policy(env, policy, n_episodes=10000):
    """Evaluate a policy by running episodes."""
    total_rewards = []
    
    for _ in range(n_episodes):
        obs, _ = env.reset()
        episode_reward = 0
        done = False
        
        while not done:
            action = np.random.choice(env.action_space.n, p=policy[obs])
            obs, reward, terminated, truncated, _ = env.step(action)
            episode_reward += reward
            done = terminated or truncated
        
        total_rewards.append(episode_reward)
    
    return total_rewards

# Evaluate both policies
env = gym.make("FrozenLake-v1", is_slippery=True)

print("Policy Comparison (10,000 episodes each)")
print("=" * 50)

rewards_random = evaluate_policy(env, pi_random, n_episodes=10000)
rewards_optimal = evaluate_policy(env, pi_star, n_episodes=10000)

print(f"\nRandom Policy:")
print(f"  Success rate: {np.mean(rewards_random)*100:.2f}%")

print(f"\nOptimal Policy:")
print(f"  Success rate: {np.mean(rewards_optimal)*100:.2f}%")

improvement = (np.mean(rewards_optimal) - np.mean(rewards_random)) / np.mean(rewards_random) * 100
print(f"\nImprovement: {improvement:.1f}% relative increase")

env.close()

In [ ]:
# Visualize the comparison
fig, ax = plt.subplots(figsize=(8, 5))

policies = ['Random Policy', 'Optimal Policy']
success_rates = [np.mean(rewards_random) * 100, np.mean(rewards_optimal) * 100]
colors = ['gray', 'green']

bars = ax.bar(policies, success_rates, color=colors, edgecolor='black')

ax.set_ylabel('Success Rate (%)')
ax.set_title('Random vs Optimal Policy on FrozenLake')
ax.set_ylim(0, 100)

for bar, rate in zip(bars, success_rates):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2,
            f'{rate:.1f}%', ha='center', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

> **Checkpoint — You should now understand:**
> - $V^*(s) = \max_\pi V^\pi(s)$ and $Q^*(s,a) = \max_\pi Q^\pi(s,a)$
> - The optimal policy is $\pi^*(s) = \arg\max_a Q^*(s,a)$
> - Bellman Optimality Equations use **max** instead of policy-weighted average
> - This non-linearity requires iterative solution methods

---
# 8. Summary and Concept Map

In this notebook, we built up the mathematical framework for reinforcement learning:

```
FROM SIMPLE TO COMPLEX: THE MDP HIERARCHY
==========================================

┌─────────────────────────────────────────────────────────────────┐
│  MARKOV PROCESS (Markov Chain)                                  │
│  ────────────────────────────                                   │
│  (S, P) - States and Transitions                                │
│  • Random walk through states                                   │
│  • No rewards, no actions                                       │
└─────────────────────────────────────────────────────────────────┘
                              ↓ Add Rewards
┌─────────────────────────────────────────────────────────────────┐
│  MARKOV REWARD PROCESS (MRP)                                    │
│  ────────────────────────────                                   │
│  (S, P, R, γ) - Add Rewards and Discount                        │
│  • Value function v(s) = E[G_t | S_t = s]                       │
│  • Bellman equation: v = R + γPv                                │
└─────────────────────────────────────────────────────────────────┘
                              ↓ Add Actions
┌─────────────────────────────────────────────────────────────────┐
│  MARKOV DECISION PROCESS (MDP)                                  │
│  ────────────────────────────                                   │
│  (S, A, P, R, γ) - The Full RL Framework                        │
│  • Policy π(a|s) defines behavior                               │
│  • V^π(s) and Q^π(s,a) value functions                          │
│  • Bellman Expectation: V = Σπ[R + γPV]                         │
│  • Bellman Optimality: V* = max[R + γPV*]                       │
└─────────────────────────────────────────────────────────────────┘


KEY EQUATIONS:
──────────────
Return:           G_t = R_{t+1} + γR_{t+2} + γ²R_{t+3} + ...

Value (MRP):      v(s) = R_s + γ Σ P_{ss'} v(s')

V^π (MDP):        V^π(s) = Σ_a π(a|s) [R_s^a + γ Σ_{s'} P_{ss'}^a V^π(s')]

Q^π (MDP):        Q^π(s,a) = R_s^a + γ Σ_{s'} P_{ss'}^a V^π(s')

V* (Optimal):     V*(s) = max_a [R_s^a + γ Σ_{s'} P_{ss'}^a V*(s')]

Optimal Policy:   π*(s) = argmax_a Q*(s,a)
```

## What's Coming Next

In the next notebook (**03_dynamic_programming.ipynb**), we'll learn algorithms to **solve** MDPs:

| Algorithm | Purpose | Requires Model? |
|-----------|---------|----------------|
| **Policy Evaluation** | Compute $V^\pi$ for a given policy | Yes |
| **Policy Iteration** | Find $\pi^*$ by alternating evaluation and improvement | Yes |
| **Value Iteration** | Find $V^*$ directly using Bellman Optimality | Yes |

These are **dynamic programming** methods that require knowing the MDP model (transitions and rewards).

---
# 9. Your Turn

Now it's time to test your understanding!

## Exercise 1: Conceptual Question

Consider the Bellman Expectation Equation for $V^\pi$:

$$V^\pi(s) = \sum_a \pi(a|s) \left[ R^a_s + \gamma \sum_{s'} P^a_{ss'} V^\pi(s') \right]$$

**Question:** Why does this equation have $V^\pi(s')$ on the right-hand side if we're trying to compute $V^\pi(s)$? Doesn't this create a circular dependency?

<details>
<summary>Click to see hint</summary>

Think about what "solving" a system of equations means. Consider a simple example with 2 states.

</details>

<details>
<summary>Click to see answer</summary>

**Yes, it's circular - and that's the point!**

The Bellman equation creates a **system of simultaneous equations**. For FrozenLake with 16 states, we have 16 equations (one per state), each relating $V(s)$ to the values of other states.

This system can be solved by:
1. **Matrix inversion**: Rearrange to $V = (I - \gamma P)^{-1} R$
2. **Iteration**: Start with a guess, repeatedly apply the Bellman update until convergence

The circular dependency is not a problem - it's what makes the equations capture the recursive nature of value!

</details>

## Exercise 2: Code Task

Verify the Bellman Optimality Equation for state 14 (adjacent to Goal).

**Task:** Compute $V^*(14)$ using the Bellman Optimality Equation and verify it matches the value from Value Iteration.

In [ ]:
# YOUR CODE HERE
# Verify Bellman Optimality Equation for state 14

# state = 14
# gamma = 0.99

# print(f"Verifying Bellman Optimality Equation for State {state}")
# print("=" * 55)
# print(f"\nBellman Optimality: V*(s) = max_a [R(s,a) + γ × Σ P(s'|s,a) × V*(s')]")

# # For each action, compute Q*(s,a) = R(s,a) + γ × Σ P(s'|s,a) × V*(s')
# q_values = []
# for a in range(4):
#     # TODO: Compute q_value for this action
#     # q_value = R_mdp[state, a] + gamma * np.sum(T_mdp[state, a] * V_star)
#     # q_values.append(q_value)
#     pass

# # V*(s) = max over actions
# # v_star_calculated = max(q_values)

# # Compare with Value Iteration result
# # print(f"\nV*(14) from Bellman: {v_star_calculated:.6f}")
# # print(f"V*(14) from Value Iteration: {V_star[14]:.6f}")

## Exercise 3: Conceptual Question

**Question:** In FrozenLake, why is the optimal policy success rate (~74%) much lower than 100%, even though we found the "optimal" policy?

<details>
<summary>Click to see hint</summary>

Think about what "optimal" means in a stochastic environment.

</details>

<details>
<summary>Click to see answer</summary>

**"Optimal" means the best possible, not perfect.**

The optimal policy maximizes the expected return, but the environment is stochastic (slippery ice). Even with the best possible action choices:

1. The agent only moves in the intended direction 1/3 of the time
2. The other 2/3, it slips perpendicular to the intended direction
3. Some slips inevitably lead to Holes

No policy can overcome the inherent randomness in the environment. The optimal policy achieves ~74% because that's the theoretical maximum given the stochastic transitions.

Compare to the deterministic case (`is_slippery=False`) where the optimal policy achieves 100% success!

</details>

---
**Congratulations! You've completed Part 2 of the RL Tutorial!**

**Key takeaways:**
- MDPs $(S, A, P, R, \gamma)$ provide the mathematical framework for RL
- Value functions $V^\pi(s)$ and $Q^\pi(s,a)$ quantify long-term desirability
- Bellman Expectation Equations relate values recursively (for a given policy)
- Bellman Optimality Equations define optimal values (using max instead of policy average)
- The optimal policy simply picks $\arg\max_a Q^*(s,a)$

**Next: 03_dynamic_programming.ipynb** - Algorithms to solve MDPs!